# Stock Compound Returns for 12 and 15 months after the start of the year
Date      : August 28, 2021                                                      
Author    : Martien Lubberink, based on a SAS script from Rabih Moussawi, see this [link](https://wrds-www.wharton.upenn.edu/pages/support/applications/risk-and-valuation-measures/stock-return-volatility-and-compound-returns-and-after-fiscal-year-ends/)

- Inputs are CRSP Monthly File and Compustat Data with FPE Dates
- Linking CRSP and Compustat using CCM 
- 12 and 15-Month Compound Returns are computed for Stocks and for the Market Index
- Computes 24-Month Total Stock Volatility, and Book Value of Equity 

You need to install the WRDS library, see this [link](https://pypi.org/project/wrds/)

Then run this once to set the pgpass file:

    import wrds
    db = wrds.Connection(wrds_username='yours')
    db.create_pgpass_file()
    
For the calcualtion of pref shares, I use [the SAS 'coalesce' equivalent for python}(https://stackoverflow.com/questions/38152389/coalesce-values-from-2-columns-into-a-single-column-in-a-pandas-dataframe)

For the mapping of returns to fiscal years, see this [sheet](https://docs.google.com/spreadsheets/d/e/2PACX-1vTn99bA4H6VWjQZmSpQ5NiK8uA_rx0ZhSdBBEOr6j_adfTqjgjtKucfTCo5kffVPzTdIHwH5v7Fe--c/pubhtml) for the logic.
 

In [ ]:
import pandas as pd
import numpy as np
import wrds
import datetime as dt

from pandas.tseries.offsets import MonthEnd
from pandas.tseries.offsets import YearEnd

db = wrds.Connection(wrds_username='xxxxxx')

In [ ]:
%matplotlib inline

In [ ]:
# Not needed, but nice for referencing variables
WrdsLib = db.list_libraries()
WComp = db.list_tables(library='comp')
WCompFunda = db.describe_table(library='comp', table='funda')
WCrspcompustatLinkTable =    db.describe_table(library='crsp', table='ccmxpf_linktable')
WCrspMsf = db.describe_table(library='crsp', table='msf')
WCrspMsi = db.describe_table(library='crsp', table='msi')
WCrspMsenames = db.describe_table(library='crsp', table='msenames')
WCompComp = db.describe_table(library='comp', table='company') # for company data sc code: sic in company
WCompFundq = db.describe_table(library='comp', table='fundq')
WCompSec = db.describe_table(library='comp', table='secm')
WCrspDelist = db.describe_table(library='crsp', table='msedelist')

In [ ]:
# Set dates
begdate = '12/31/2015'
enddate = '12/31/2020'

In [ ]:
#%% It takes a while to get the CCM file
def ccm_data():
    ccm_m = db.raw_sql("""
    select a.gvkey, a.datadate, b.lpermno as permno, b.lpermco as permco, b.LINKENDDT, b.LINKDT, b.LINKTYPE
    from comp.funda a,
    crsp.ccmxpf_linktable as b
    where a.gvkey=b.gvkey
    and a.datafmt = 'STD' and a.consol = 'C' and a.indfmt = 'INDL'
    and b.linkprim in ('P', 'C')
    and b.LINKTYPE in ('LU', 'LC')
    and a.datadate >= b.LINKDT and (a.datadate <= b.LINKENDDT or b.LINKENDDT IS NULL)
    """)
    ccm_m[['datadate',  'linkenddt', 'linkdt']] = ccm_m[['datadate',  'linkenddt', 'linkdt']].apply(pd.to_datetime, errors='coerce')
    ccm_m.dropna(subset = ['gvkey', 'permno','permco', 'datadate'], inplace= True)
    ccm_m[['permno','permco']] = ccm_m[['permno','permco']].astype(int)
    ccm_m.set_index(['gvkey','datadate'], inplace= True)
    return ccm_m

ccm_m = ccm_data()

In [ ]:
# Retrieve a whole bunch of accounting data. Note that I filter on ni, ceq, sale, fic

def annual_acc__data(begdate, ccm_m):
    #  'year' will be based on datadate
    df_ann = db.raw_sql("""
                        select gvkey,datadate,pstkrv,pstkl,pstk,at,ceq,cstk,re,seq,txdb,itcb,conm,cik,exchg,fyr,fic,ci,csho,teq,ni,fyear,sale,caps,tstk,tstkc,tstkp,tstkn,citotal,dvc,dvp
                        from comp.funda
                        where indfmt='INDL'
                        and datafmt='STD'
                        and popsrc='D'
                        and consol='C'
                        and fic='USA'
                        and ni<>0
                        and ceq<>0
                        and sale>0
                        and datadate >= '%(begdate)s'
                        """ % {'begdate' : begdate})
    df_ann['datadate'] = pd.to_datetime(df_ann['datadate'])  # convert datadate to date format
    df_ann['year'] = df_ann['datadate'].dt.year

    # prefs 
    df_ann['pref'] = df_ann[['pstkrv', 'pstkl', 'pstk']].bfill(axis=1).iloc[:, 0].mul(-1)

    # book equity
    df_ann['be'] = df_ann[['seq', 'itcb', 'pstk', 'pref']].sum(axis=1)
    df_ann.set_index(["gvkey", "datadate"], inplace=True)

    # merge with CCM
    df_ann = df_ann.join(ccm_m[['permno', 'permco']])
    df_ann.dropna(subset=['permno', 'permco'], inplace=True)
    df_ann[['permno', 'permco']] = df_ann[['permno', 'permco']].astype(int)
    return df_ann  # [['conm', 'year', 'fyear', 'fyr', 'at', 'ni', 'sale', 'be','caps', 'tstk', 'tstkc', 'tstkp', 'tstkn', 'citotal', 'dvc', 'dvp','pstkrv', 'pstkl', 'pstk', 'ceq', 'cstk', 're', 'seq', 'txdb', 'itcb', 'cik', 'exchg', 'fic', 'ci', 'csho', 'teq']]

df_ann = annual_acc__data(begdate, ccm_m)

In [ ]:
df_ann

In [ ]:
# Monthly returns and some other vars for calculation of share price and tso, see below

def monthly_returns(begdate, enddate):
    #  'year' will be based on datadate
    dfm = db.raw_sql("""
                     select a.permco, a.permno, a.date, a.ret, a.prc, a.cfacpr, a.shrout, a.cfacshr
                     from crsp.msf as a, crsp.msenames as b
                     where a.date between '%(begdate)s' and '%(enddate)s'
                     and a.permno=b.permno
                     and b.namedt<=a.date
                     and a.date<=b.nameendt
                     and b.shrcd in (10,11)
                     order by permno, date
                     """ % {'begdate' : begdate, 'enddate' : enddate})
    dfm['datadate'] = pd.to_datetime(dfm['date']) + MonthEnd(0)
    dfm['permno'] = dfm['permno'].astype(int)
    dfm['year'] = dfm['datadate'].dt.year
    dfm.set_index(['permno', 'datadate'], inplace=True)
    return dfm

In [ ]:
dfm = monthly_returns(begdate, enddate)

In [ ]:
dfm

In [ ]:
# Monthly index returns
def index_returns(begdate, enddate, dfm):
    dfi = db.raw_sql("select date, vwretd from crsp.msi where date between '%(begdate)s' and '%(enddate)s'" % {'begdate' : begdate, 'enddate' : enddate})
    dfi['datadate'] = pd.to_datetime(dfi['date']) + MonthEnd(0)
    dfi.set_index(['datadate'], inplace=True)
    return dfm.swaplevel().join(dfi['vwretd']).swaplevel()

dfm = index_returns(begdate, enddate, dfm)

In [ ]:
dfm

In [ ]:
# Monthly Delisting returns - the original returns are kept: org_ret
def delisting_returns(begdate, enddate, dfm):
    #  year will be based on datadate
    dfi = db.raw_sql("select permno, dlstdt, dlret from crsp.msedelist where dlstdt between '%(begdate)s' and '%(enddate)s'" % {'begdate' : begdate, 'enddate' : enddate})
    dfi['datadate'] = pd.to_datetime(dfi['dlstdt']) + MonthEnd(0)
    dfi.set_index(['permno', 'datadate'], inplace=True)
    # dfi['year']   = dfi.index.get_level_values(0).year
    dfn = dfm.swaplevel().join(dfi['dlret']).swaplevel()
    df_up = dfn[['ret', 'dlret']].copy()
    df_up += 1
    df_up['dl_ret'] = df_up.prod(axis=1)
    df_up -= 1
    dfn = dfn.join(df_up['dl_ret'])
    dfn.rename(columns={'dl_ret': 'ret', 'ret': 'org_ret'}, inplace=True)
    return dfn
dfm = delisting_returns(begdate, enddate, dfm)

In [ ]:
testm = dfm.loc[dfm.permco ==  20000]
testm

In [ ]:
# Assigning returns to the proper time-period

def align_fyr(dfm):  
    # join returns with 'fyr', 'permco', 'year' and set index on permco and year
    dfs = dfm.reset_index().set_index(['permco', 'year']).join(df_ann[['fyr', 'permco', 'year']].reset_index()[['gvkey', 'fyr', 'permco', 'year']].set_index(['permco', 'year']))
    # make a dict of periods and fiscal year end numbers
    months = ['A-' + x.upper() for x in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]
    months_dic = dict(zip(months, list(range(1, 13))))
    dfr = pd.DataFrame()
    for key, value in months_dic.items():
        print(key)
        # Select obs for a fyr value [1-12]
        dfp = dfs.loc[dfs['fyr'] == value, ['datadate', 'ret', 'vwretd']].copy()
        # Convert the period into an integer, not elegant, but it works
        dfp['period'] = pd.PeriodIndex(dfp['datadate'], freq=key).astype(str).astype(int)
        dfp = dfp.reset_index().set_index(['permco', 'period', 'datadate'])
        # Determine the returns
        dfp[['ret', 'vwretd']] += 1
        dfp = dfp.groupby(['permco',  'period'])[['ret', 'vwretd']].prod()
        dfp[['ret', 'vwretd']] -= 1
        dfr = dfr.append(dfp)
    dfr.sort_index(level=['permco', 'period'], inplace=True)
    dfr.rename(columns={'period': 'year'}, inplace=True)
    # year now replaces period
    dfr.index.names = ['permco', 'year']
    return dfr


df1 = align_fyr(dfm)
test_firm1 = df1.loc[[5,357,7, 20000]]
test_firm1

In [ ]:
# Assigning returns from three months after the fiscal year to the prior year to obtain 3-months post FYE return

def add_quarters(dfm, yearback):
    dfs = dfm.reset_index().set_index(['permco', 'year']).join(df_ann[['fyr', 'permco', 'year']].reset_index()[['gvkey', 'fyr', 'permco', 'year']].set_index(['permco', 'year']))
    # year is based on datadate
    quarters = ['Q-' + x.upper() for x in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]
    quarters_dic = dict(zip(quarters, list(range(1, 13))))
    dfr = pd.DataFrame()
    for key, value in quarters_dic.items():
        print(key)
        dfp = dfs.loc[dfs['fyr'] == value, ['datadate', 'ret', 'vwretd']].copy()
        dfp['quarter'] = dfp['datadate'].dt.to_period(key).astype(str)  
        dfp['period_from_quarter'] = dfp['quarter'].str[:4].astype(int) # Extract year from period
        dfp['quarter_num'] = dfp['quarter'].str[-1:].astype(int) # Extract quarter number from period
        # obtain only 1st quarter
        dfp = dfp.loc[dfp['quarter_num'] == 1].copy()
        dfp = dfp.reset_index().set_index(['permco', 'period_from_quarter', 'datadate'])
        # Determine the returns
        dfp[['ret', 'vwretd']] += 1
        dfp = dfp.groupby(['permco',  'period_from_quarter'])[['ret', 'vwretd']].prod()
        dfp[['ret', 'vwretd']] -= 1
        dfr = dfr.append(dfp)
    dfr.sort_index(level=['permco', 'period_from_quarter'], inplace=True)
    dfr['year'] = dfr.index.get_level_values(1)
    # Make sure the 3 monts returns matches with the prior year
    dfr['year'] -= yearback
    dfr.rename(columns={'ret': 'ret90', 'vwretd': 'vwretd90'}, inplace=True)
    dfr = dfr.reset_index().set_index(['permco', 'year'])
    return dfr


df2 = add_quarters(dfm, 1)

test_firm2 = df2.loc[20000]
test_firm2

In [ ]:
#%% Merge the two return sets to obtain 15-months return
def add_90_days_returns(df1, df2, skipper):
    # skipper determines if we can compound over both periods if the last period is missing
    dfn = df1.join(df2[['ret90', 'vwretd90']])
    df_up = dfn.copy()
    df_up += 1
    df_up['ret15']   = df_up[['ret',    'ret90']].prod(axis=1, skipna=skipper)
    df_up['vwret15'] = df_up[['vwretd', 'vwretd90']].prod(axis=1, skipna=skipper)
    df_up -= 1
    dfn = dfn.join(df_up[['ret15', 'vwret15']])
    return dfn


df_full_returns = add_90_days_returns(df1, df2, False)

test_firm3 = df_full_returns.loc[20000]

test_firm3

In [ ]:
# Merge with accounting data to obtain end result

df_merged = df_ann.reset_index().merge(df_full_returns.reset_index(), on = ['permco', 'year']).set_index(['gvkey', 'datadate'])

In [ ]:
# Diognostics here
# Show returns histo
df_merged.ret.clip(upper=5.0).hist(bins = 100)
test_all = df_merged.loc[df_merged.permco ==  20000]
test_all

In [ ]:
#%% Nice to have tools - get the accounting for MSFT
def retrieve_accounting_data(firm):
    df = db.raw_sql("""select gvkey, datadate, at, re
               from comp.funda
               where gvkey = '%(gvkey)s'
               """ % {'gvkey' : firm})
    df['datadate']=pd.to_datetime(df['datadate']) #convert datadate to date fmt
    df.set_index(["gvkey", "datadate"], inplace= True)
    return df

msft_acc = retrieve_accounting_data('012141')
msft_acc

In [ ]:
#%% Nice to have tools - get the accounting for MSFT
def retrieve_price_data(firm):
    df = db.raw_sql("""select permco, permno, date, ret, prc, cfacpr, shrout, cfacshr
               from crsp.msf
               where permco = '%(permco)s'
               """ % {'permco' : str(firm)})
    df['datadate']=pd.to_datetime(df['date']) #convert datadate to date fmt
    df.set_index(["permco", "datadate"], inplace= True)
    df =  df.assign(price = lambda x: abs(x['prc'])/df['cfacpr'],
                    tso =   lambda x: x['shrout']*x['cfacshr']*1000,
                    mv =    lambda x: x['price']*x['tso']/1000000
                    )
    return df

In [ ]:
msftp = retrieve_price_data(8048)
msftp.mv.plot()


In [ ]:
msftp.reset_index(level = 'permco').mv.plot()

In [ ]:
dfm.loc[dfm.permco==8048, 'prc'].plot()